## Flight data record scrapping 

In [4]:
# Imports
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By

import pandas as pd
import string
import csv

In [2]:
# Initializing the browser
path_to_chromedriver = '/home/bt/Documents/Workplace/going_headless/chromedriver/chromedriver'
browser = webdriver.Chrome(executable_path = path_to_chromedriver)

In [3]:
## Get the website 
url = 'https://www.epower.amadeus.com/biscord/#AdtCount=1&CabinClass=&Culture=en-US&DepartureDate=1/17/2018&From=LOS&Method=Search&Page=Result&ProviderList=OnlyAmadeus&ReturnDate=1/30/2018&To=LON'
browser.get(url)

TimeoutException: Message: timeout
  (Session info: chrome=62.0.3202.75)
  (Driver info: chromedriver=2.35.528139 (47ead77cb35ad2a9a83248b292151462a66cd881),platform=Linux 4.10.0-19-generic x86_64)


In [7]:
# waiting things
delay = 7 # seconds
xpaths = '//*[@id="Obj_Result"]/div/div[3]/div'
try:
    element_present = EC.presence_of_element_located((By.XPATH, xpaths))
    WebDriverWait(browser, timeout).until(element_present)
    print "Page is ready!"
except TimeoutException:
    print "Loading took too much time!"

### Functions

In [8]:
# 1. Check if element exist
def elements_exist_by_xpath(xpath):
    try:
        elm = browser.find_elements_by_xpath(xpath)
        #elm.click()
    except NoSuchElementException:
        return False
    return True

# 2. Check if xpath element is visibe/availiabe
def is_available(xpath):
    try:
        element = wait.until(EC.visibility_of_element_located,xpath)
    except:
        return False
    return True

In [10]:
# Results panal
xpaths = '//*[@id="Obj_Result"]/div/div[3]/div'

In [12]:
# Robust Wrapping
# Check if the path is available
if is_available(xpaths):
    # check elements existence
    if elements_exist_by_xpath(xpaths):
        #print("...Results Present")
        # Get all result element
        print('...acquiring reseult panal element')
        all_elem = browser.find_elements_by_xpath(xpaths)
        # Check if the list isn't empty
        if len(all_elem) > 0:
            print("...{} Result Element acquired".format(len(all_elem)))
        else:
            print("...{} Result Element acquired".format(len(all_elem)))
else:
    print('No Result Panel')

...acquiring reseult panal element
...11 Result Element acquired


### Function 2

In [13]:
# Extract data from all result element
def get_item_by_xpath(all_res_elem,xpath):
    result = []
    for elm in all_res_elem[:-1]:
        if elm.text != ' ':
            item = elm.find_element_by_xpath(xpath).text
            result.append(item)
    return(result)

def get_item_by_xpath2(all_res_elem,xpath):
    result = []
    for elm in all_res_elem[:-1]:
        if elm.text != ' ':
            try:
                item = elm.find_element_by_xpath(xpath).text
                result.append(item)
            except NoSuchElementException:
                try:
                    item = elm.find_element_by_xpath('div/'+xpath).text
                    result.append(item)
                except NoSuchElementException:
                    try:
                        item = elm.find_element_by_xpath('div/div/'+xpath).text
                        result.append(item)
                    except NoSuchElementException:
                        try:
                            item = elm.find_element_by_xpath('div/div/div/'+xpath).text
                            result.append(item)
                        except NoSuchElementException:
                            result.append('not available')
                
    return(result)

## Data Acquisition

In [14]:
# Data
# Airline list
airline_xpath = 'div[1]/div/div/div[1]/div/div[1]'
airline_list = get_item_by_xpath(all_elem,airline_xpath)
#airline_list
# Price list
price_xpath = 'div[1]/div/div/div[3]/div[2]/div/div/div[1]/div[2]'
price_list = get_item_by_xpath(all_elem,price_xpath)
price_list
# Number of seats left list
no_seat_left_xpath = 'div[1]/div/div/div[3]/div[2]/div/div/div[1]/div[1]/label'
no_seats_left_list = get_item_by_xpath(all_elem,no_seat_left_xpath)
no_seats_left_list
# Time of Travel list
time_traveld_xpath = 'div/div/div/div[1]/div/div[2]/div[2]/div[2]/div/div[2]/span'
time_traveld_list = get_item_by_xpath2(all_elem,time_traveld_xpath)
#time_traveld_list
# arrival
arr_time_xpath = 'div[1]/div/div/div[1]/div/div[2]/div[2]/div[2]/div/div[3]/div/div/div'
arr_time_list = get_item_by_xpath2(all_elem,arr_time_xpath)
#arr_time_list
# Destination info
to_loc_xpath = 'div[1]/div/div/div[1]/div/div[2]/div[2]/div[2]/div/div[3]/div/div/span[1]'
to_loc_list = get_item_by_xpath2(all_elem,to_loc_xpath)
#to_loc_list
# depture_time
dep_time_xpath = 'div[1]/div/div/div[1]/div/div[2]/div[2]/div[2]/div/div[1]/div/div/div'
dep_time_list = get_item_by_xpath2(all_elem,dep_time_xpath)
#dep_time_list
# from
from_xpath = 'div[1]/div/div/div[1]/div/div[2]/div[2]/div[2]/div/div[1]/div/div/span[1]'
from_list = get_item_by_xpath2(all_elem,from_xpath)
#from_list
# Number of stops
num_stops_xpath = 'div[1]/div/div/div[1]/div/div[2]/div[2]/div[2]/div/div[2]/div'
num_stops_list = get_item_by_xpath2(all_elem,num_stops_xpath)
#num_stops_list

## Querry Ends

## Data Processing and Structuring

In [16]:
# Processes Price
price1_list = []
price2_list = []
play = price_list

for p in play:
    a = p.split('\n')
    price1_list.append(a[0])
    if len(a)>1:
        price2_list.append(a[1])
    else:
        price2_list.append("")

In [17]:
# Structuring the data
col_name =['Airline','From','Dep_time','To','Arr_time','Num_stops','No_seats','Price1','Price2']
structured_dict = {'Airline':airline_list,'From':from_list,
                   'Dep_time':dep_time_list,'To':to_loc_list,
                   'Arr_time':arr_time_list,'Num_stops':num_stops_list,
                   'Num_seats':no_seats_left_list,'Price1':price1_list,'Price2':price2_list}
#pd.DataFrame(price_list)

In [18]:
str_data = pd.DataFrame(structured_dict)

In [19]:
str_data

,Airline,Arr_time,Dep_time,From,Num_seats,Num_stops,Price1,Price2,To
0,Ethiopian Airlines,6:35am,11:10am,LOS,4 seat(s) left,2 Stop(s),"318,653.00 NGN","307,060.64 NGN",LHR
1,Qatar Airways,6:20am,12:00pm,LOS,5 seat(s) left,1 Stop DOH,"328,106.00 NGN","321,218.73 NGN",LHR
2,Etihad Airways,12:05pm,9:40am,LOS,7 seat(s) left,1 Stop AUH,"323,107.00 NGN",,LHR
3,Emirates,11:40am,6:50pm,LOS,9 seat(s) left,1 Stop DXB,"365,722.00 NGN","355,215.98 NGN",LGW
4,Egyptair,12:35pm,2:25pm,LOS,6 seat(s) left,1 Stop CAI,"375,134.00 NGN",,LHR
5,Turkish Airlines,4:15pm,10:40pm,LOS,9 seat(s) left,1 Stop IST,"403,148.00 NGN","385,929.75 NGN",LHR
6,Lufthansa,8:40am,11:50pm,LOS,1 seat left,1 Stop FRA,"405,702.00 NGN",,LHR
7,Royal Air Maroc,not available,not available,not available,9 seat(s) left,not available,"409,546.00 NGN",,not available
8,British Airways,4:55am,11:25pm,LOS,1 seat left,Direct,"629,893.00 NGN",,LHR
9,Virgin Atlantic,not available,not available,not available,2 seat(s) left,not available,"1,062,674.00 NGN",,not available


In [20]:
str_data.to_csv('fligthRec.csv',header=True,index=False,columns=col_name)

In [21]:
browser.close()